## Purpose of first model
The purpose of the FINAL model is to do topic modeling of both hits in the US and the UK and comparing them.
However, this first, simple, prelimiary model will do just the US, dropping the NaN rows (currently trying to find ways to fill them in the webscraper module).



In [1]:
import pickle
import pandas as pd

import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
file = open('../Webscraper/uk_pops_raw', 'rb')
uk_pops = pickle.load(file)
uk_pops.dropna(inplace=True)
uk_pops = uk_pops['Lyrics']

In [3]:
uk_pops

0       0-9\n10 Years\n100 Demons\n1000 Homo DJs\n12 S...
1       So you want\nTo be free?\nTo live your life\nT...
2       Whoa, my love\nMy darling\nI've hungered for y...
3       It's been seven hours and fifteen days\nSince ...
4       There you go\nFlashing fever from your eyes\nH...
                              ...                        
2793    Te prometo que nunca encontrarás a alguien igu...
2794    Baby when it comes to love, it should be mutua...
2795    Searchin' every corner of my mind\nLookin' for...
2796    Ik bel je één keer, twee keer‚ drie keer\nIk  ...
2797    All my life I've been a good girl\nTryna do wh...
Name: Lyrics, Length: 2641, dtype: object

In [4]:
# only keep alphanumeric, remove newlines, and make everything lowercase
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
rm_newline = lambda x: re.sub("\n", ' ',x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

uk_pops_nopunc = uk_pops.map(alphanumeric).map(rm_newline).map(punc_lower)

In [5]:
# lemmatize. do this by turning every line into a list of words, and concat back to full lyrics
uk_pops_list = uk_pops_nopunc.apply(lambda x: x.split(' '))

def lemmatize(list_of_words):
    lmt = WordNetLemmatizer()
    strg = ''
    for word in list_of_words:
        lem_word = lmt.lemmatize(word)
        if len(lem_word) > 2:
            strg += lem_word + ' '
    return strg
        

uk_pops_clean = uk_pops_list.apply(lemmatize)

In [6]:
cv = CountVectorizer(
    stop_words = 'english',
    max_df = 0.2,
    min_df = 0.005
)
X = cv.fit_transform(uk_pops_clean)
doc_term_matrix = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())

In [7]:
doc_term_matrix

,aah,abandoned,ability,able,abode,abraham,abroad,absence,absent,absolute,...,york,young,younger,youth,youthful,yup,zero,zombie,zone,zoo
0,0,1,0,0,0,0,0,0,0,0,...,0,3,0,0,0,0,2,2,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2637,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2638,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
2639,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.feature_extraction import text

In [9]:
stop_words_added = text.ENGLISH_STOP_WORDS.union(['huh','woo','whoa','hey','hold','ooh','aah','whoa','oooh','ayy','hoo'])
# stop_words_custom = frozenset({'huh','woo','whoa','hey','hold'})

In [50]:
tfidf =  TfidfVectorizer(
    stop_words = stop_words_added,
    max_df = 0.2,
    min_df = 0.005
)
Y = tfidf.fit_transform(uk_pops_clean)
doc_term_matrix2 = pd.DataFrame(Y.toarray(), columns=tfidf.get_feature_names())


In [51]:
doc_term_matrix2

,abandoned,ability,able,abode,abraham,abroad,absence,absent,absolute,absolutely,...,york,young,younger,youth,youthful,yup,zero,zombie,zone,zoo
0,0.016056,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.027720,0.0,0.000000,0.0,0.0,0.030414,0.032713,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2636,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2637,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
2638,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.042739,0.0,0.060965,0.0,0.0,0.000000,0.000000,0.0,0.0
2639,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


### LSA below

In [12]:
lsa = TruncatedSVD(5)
doc_topic = lsa.fit_transform(doc_term_matrix2)
lsa.explained_variance_ratio_

array([0.00435737, 0.00682388, 0.00612271, 0.00514681, 0.00487399])

In [13]:
topic_word = pd.DataFrame(
    lsa.components_.round(3), 
    index = ['topic1','topic2','topic3','topic4','topic5'],
    columns = tfidf.get_feature_names()
    )
topic_word

,abandoned,ability,able,abode,abraham,abroad,absence,absent,absolute,absolutely,...,york,young,younger,youth,youthful,yup,zero,zombie,zone,zoo
topic1,0.002,0.002,0.010,0.001,0.001,0.002,0.002,0.001,0.001,0.003,...,0.015,0.050,0.004,0.005,0.001,0.003,0.003,0.002,0.011,0.002
topic2,-0.002,-0.002,-0.008,-0.001,-0.001,-0.002,-0.001,-0.001,-0.001,-0.002,...,-0.001,-0.018,-0.003,-0.004,-0.001,0.000,-0.001,0.003,0.012,-0.001
topic3,0.001,-0.001,-0.002,0.000,-0.000,-0.000,-0.000,-0.000,-0.001,-0.000,...,-0.020,-0.015,-0.000,-0.001,-0.000,-0.004,-0.003,-0.004,-0.020,-0.002
topic4,-0.002,-0.002,-0.008,-0.001,-0.000,-0.002,0.000,-0.001,-0.001,-0.003,...,-0.002,-0.027,-0.003,-0.005,-0.000,0.000,0.002,-0.004,-0.010,-0.001
topic5,0.000,0.002,0.006,0.001,0.001,0.001,-0.000,0.000,0.001,0.002,...,0.011,0.016,0.002,0.003,0.000,0.001,0.002,-0.000,0.001,0.002


In [14]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [15]:
display_topics(lsa, cv.get_feature_names(), 10)


Topic  0
toll, listened, bowl, ham, gossip, lied, stock, tryna, logic, reader

Topic  1
toll, damsel, paris, eventually, bob, stock, aloud, flock, dancer, cloud

Topic  2
toll, starting, golden, dread, faithful, foreign, hollow, los, skeleton, bein

Topic  3
bowl, gossip, toll, kind, torn, feeding, crawl, background, bet, reader

Topic  4
pushing, pool, panel, toll, stock, newly, starting, coloured, looked, shield


### NMF below

In [58]:
nmf_model = NMF(7)
doc_topic2 = nmf_model.fit_transform(doc_term_matrix2)

In [59]:
topic_word = pd.DataFrame(
    nmf_model.components_.round(3), 
    columns = tfidf.get_feature_names()
    )
topic_word

,abandoned,ability,able,abode,abraham,abroad,absence,absent,absolute,absolutely,...,york,young,younger,youth,youthful,yup,zero,zombie,zone,zoo
0,0.006,0.000,0.001,0.000,0.001,0.000,0.001,0.000,0.000,0.00,...,0.000,0.015,0.004,0.000,0.000,0.003,0.000,0.000,0.000,0.000
1,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.000,0.017,0.000,0.000,0.000,0.000,0.001,0.001,0.003,0.000
2,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.010,0.000,0.000,0.000,0.000,0.005,0.000,0.012,0.046,0.000
3,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.000,0.000,0.00,...,0.017,0.002,0.000,0.000,0.000,0.003,0.005,0.000,0.000,0.000
4,0.000,0.000,0.001,0.000,0.000,0.000,0.002,0.000,0.000,0.00,...,0.000,0.004,0.000,0.001,0.000,0.001,0.000,0.000,0.000,0.000
5,0.005,0.013,0.063,0.007,0.006,0.012,0.008,0.005,0.011,0.02,...,0.085,0.286,0.017,0.036,0.004,0.009,0.018,0.011,0.046,0.014
6,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.00,...,0.006,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [60]:
display_topics(nmf_model, tfidf.get_feature_names(), 10)


Topic  0
gone, dream, light, stay, home, long, feeling, turn, believe, leave

Topic  1
tonight, stay, tomorrow, hand, party, going, feelin, dancing, high, kiss

Topic  2
dance, everybody, body, party, stop, music, floor, rock, beat, rhythm

Topic  3
boy, gotta, bad, kiss, bring, crazy, better, walk, touch, sweet

Topic  4
little, bit, crazy, bad, late, maybe, conversation, work, fun, broken

Topic  5
shit, nigga, people, fuck, woman, money, black, head, bitch, bad

Topic  6
que, para, por, com, los, vida, est, stay, dale, sur


### LDA Below

In [19]:
# # for TF DTM
# lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tf.fit(doc_term_matrix)

# # for TFIDF DTM
# lda_tfidf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tfidf.fit(doc_term_matrix2)